## Custom API Notebook

### Initialize the Server

In [ ]:
# stdlib
from typing import Any

# syft absolute
import syft as sy
from syft import SyftError
from syft import SyftSuccess

server = sy.orchestra.launch(
    name="test-datasite-1",
    dev_mode=True,
    create_producer=True,
    n_consumers=3,
    reset=True,
)

datasite_client = server.login(email="info@openmined.org", password="changethis")
datasite_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
datasite_guest = server.login(email="user@openmined.org", password="verysecurepassword")

### Create a public custom API Endpoint by using the decorator

This allows server admin to create a new public endpoint by using only the decorator.

In [ ]:
@sy.api_endpoint(
    path="first.query",
    settings={"key": "value"},
)
def public_endpoint_method(
    context,
    query: str,
) -> "Any":
    return context.settings["key"] == "value"


# Add it to the server.
response = datasite_client.api.services.api.add(endpoint=public_endpoint_method)
response

In [ ]:
assert isinstance(response, SyftSuccess)

In [ ]:
datasite_client.api.services.api.api_endpoints()

In [ ]:
assert len(datasite_client.api.services.api.api_endpoints()) == 1

In [ ]:
# Once api refresh is done, remove this cell
datasite_client = server.login(email="info@openmined.org", password="changethis")
datasite_guest = server.login(email="user@openmined.org", password="verysecurepassword")

In [ ]:
assert datasite_client.api.services.first.query(query="SELECT *")

In [ ]:
result = datasite_guest.api.services.first.query(query="SELECT *")
result

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [ ]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def public_function(
    context,
) -> str:
    return "Public Function Execution"


@sy.api_endpoint_method(settings={"Hello": "Private"})
def private_function(
    context,
) -> str:
    return "Private Function Execution"


new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    mock_function=public_function,
    private_function=private_function,
    description="Lore ipsulum ...",
)

# # Add it to the server.
response = datasite_client.api.services.api.add(endpoint=new_endpoint)

In [ ]:
datasite_client.api.services.api.api_endpoints()

In [ ]:
assert isinstance(response, SyftSuccess)
assert len(datasite_client.api.services.api.api_endpoints()) == 2

In [ ]:
# Once api refresh is done, remove this cell
datasite_client.refresh()
datasite_guest.refresh()

In [ ]:
datasite_client.api.services.third.query()

In [ ]:
assert datasite_client.api.services.third.query() == "Private Function Execution"

In [ ]:
assert datasite_guest.api.services.third.query() == "Public Function Execution"

In [ ]:
datasite_guest.api.services.third.query()

In [ ]:
@sy.syft_function_single_use(
    endpoint=datasite_guest.api.services.third.query,
)
def job_function(endpoint):
    return endpoint()


# Create a new project
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[datasite_guest],
)

result = new_project.create_code_request(job_function, datasite_guest)
assert isinstance(result, SyftSuccess)

In [ ]:
res = None
for r in datasite_client.requests.get_all():
    if r.requesting_user_email == "user@openmined.org":
        res = r.approve()

assert res is not None, res
res

In [ ]:
result = datasite_guest.code.job_function(
    endpoint=datasite_client.api.services.third.query
)
result

In [ ]:
assert not isinstance(result, SyftError), result

In [ ]:
assert result.get() == "Private Function Execution"

In [ ]:
with sy.raises(sy.SyftException, show=True):
    datasite_guest.api.services.third.query.private()

In [ ]:
result = datasite_client.api.services.api.delete(endpoint_path="third.query")
assert isinstance(result, SyftSuccess)
result

In [ ]:
assert len(datasite_client.api.services.api.api_endpoints()) == 1

## Updating Endpoints

First we'll create a new endpoint

In [ ]:
@sy.api_endpoint(
    path="test.update",
    settings={"key": "value"},
)
def new_public_function(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the server.
response = datasite_client.api.services.api.add(endpoint=new_public_function)

assert isinstance(response, SyftSuccess), response
response

#### Update the public function

In [ ]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def updated_public_function(
    context,
) -> str:
    return "Updated Public Function Execution"


response = datasite_client.api.services.api.update(
    endpoint_path="test.update", mock_function=updated_public_function
)
assert isinstance(response, SyftSuccess), response
response

#### Update the private function

In [ ]:
@sy.api_endpoint_method(settings={"Hello": "Private"})
def updated_private_function(
    context,
) -> str:
    return "Updated Private Function Execution"


response = datasite_client.api.services.api.update(
    endpoint_path="test.update", private_function=updated_private_function
)
assert isinstance(response, SyftSuccess), response
response

#### Update both functions with a pair that has a new signature

In [ ]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def new_sig_public_function(context, new_parameter) -> str:
    return "Updated Public Function Execution"


@sy.api_endpoint_method(settings={"Hello": "Private"})
def new_sig_private_function(context, new_parameter) -> str:
    return "Updated Private Function Execution"


response = datasite_client.api.services.api.update(
    endpoint_path="test.update",
    mock_function=new_sig_public_function,
    private_function=new_sig_private_function,
)
assert isinstance(response, SyftSuccess), response
response

### Invalid update attempts
- Both functions empty
- Signature mismatch
- Non existing endpoint

#### Both functions are empty

In [ ]:
with sy.raises(sy.SyftException, show=True):
    response = datasite_client.api.services.api.update(endpoint_path="test.update")

#### Signature mismatch

In [ ]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def bad_public_function(context, foo) -> str:
    return "Updated Public Function Execution"


with sy.raises(sy.SyftException, show=True):
    response = datasite_client.api.services.api.update(
        endpoint_path="test.update", mock_function=bad_public_function
    )

Non Existing endpoint

In [ ]:
with sy.raises(sy.SyftException, show=True):
    response = datasite_client.api.services.api.update(
        endpoint_path="nonexistent", mock_function=bad_public_function
    )

# Syft Function/API Logs

In [ ]:
@sy.api_endpoint_method()
def public_log_function(
    context,
) -> str:
    print("Logging Public Function Call")
    return "Public Function Execution"


@sy.api_endpoint_method()
def private_log_function(
    context,
) -> str:
    print("Logging Private Function Call")
    return "Private Function Execution"


new_endpoint = sy.TwinAPIEndpoint(
    path="test.log",
    mock_function=public_log_function,
    private_function=private_log_function,
    description="Lore ipsulum ...",
)

# # Add it to the server.
response = datasite_client.api.services.api.add(endpoint=new_endpoint)

In [ ]:
@sy.syft_function_single_use(endpoint=datasite_client.api.services.test.log)
def test_log_call(endpoint):  # noqa: F811
    print("In Syft Function Context")
    endpoint()
    print("After API endpoint call")
    return True


@sy.syft_function_single_use(endpoint=datasite_client.api.services.test.log)
def test_log_call_mock(endpoint):  # noqa: F811
    print("In Syft Function Context")
    endpoint.mock()
    print("After API endpoint call")
    return True


@sy.syft_function_single_use(endpoint=datasite_client.api.services.test.log)
def test_log_call_private(endpoint):  # noqa: F811
    print("In Syft Function Context")
    endpoint.private()
    print("After API endpoint call")
    return True


# Create a project
project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[datasite_client],
)
project.create_code_request(test_log_call, datasite_client)
project.create_code_request(test_log_call_mock, datasite_client)
project.create_code_request(test_log_call_private, datasite_client)

In [ ]:
log_call_job = datasite_client.code.test_log_call(
    endpoint=datasite_client.api.services.test.log, blocking=False
)

In [ ]:
log_call_mock_job = datasite_client.code.test_log_call_mock(
    endpoint=datasite_client.api.services.test.log, blocking=False
)

In [ ]:
log_call_private_job = datasite_client.code.test_log_call_private(
    endpoint=datasite_client.api.services.test.log, blocking=False
)

In [ ]:
# stdlib
import time

# syft absolute
from syft.service.job.job_stash import JobStatus

# Iterate over the Jobs waiting them to finish their pipelines.
job_pool = [
    (log_call_job, "Logging Private Function Call"),
    (log_call_mock_job, "Logging Public Function Call"),
    (log_call_private_job, "Logging Private Function Call"),
]
for job, expected_log in job_pool:
    updated_job = datasite_client.api.services.job.get(job.id)
    while updated_job.status in {JobStatus.CREATED, JobStatus.PROCESSING}:
        updated_job = datasite_client.api.services.job.get(job.id)
        time.sleep(1)

    assert (
        updated_job.status == JobStatus.COMPLETED
    ), f"Job {updated_job.id} exited with status {updated_job.status} and result {updated_job.result}"
    if updated_job.status == JobStatus.COMPLETED:
        print(f"Job {updated_job.id} completed")
        # If they're completed. Then, check if the TwinAPI print appears in the job logs.
        assert expected_log in datasite_client.api.services.job.get(job.id).logs(
            _print=False
        )